### Feature Extractors
Extraction: Extracting features from “raw” data
+ TF-IDF
+ Word2Vec
+ CountVectorizer
+ FeatureHasher

##### Tf-IDF

In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("Feature Extractor").getOrCreate()

22/03/29 21:39:29 WARN Utils: Your hostname, iamhimanshu0 resolves to a loopback address: 127.0.1.1; using 192.168.43.239 instead (on interface wlo1)
22/03/29 21:39:29 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/03/29 21:39:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer

In [4]:
sentenceData = spark.createDataFrame([
    (0.0, "Hi I heard about Spark"),
    (0.0, "I wish Java could use case classes"),
    (1.0, "Logistic regression models are neat")
], ["label", "sentence"])

sentenceData.show()

+-----+--------------------+
|label|            sentence|
+-----+--------------------+
|  0.0|Hi I heard about ...|
|  0.0|I wish Java could...|
|  1.0|Logistic regressi...|
+-----+--------------------+



In [5]:
tokenizer = Tokenizer(inputCol='sentence', outputCol='words')
wordsData = tokenizer.transform(sentenceData)

In [6]:
hashingTF = HashingTF(inputCol='words', outputCol='rawFeatures',numFeatures=20)
featurizedData = hashingTF.transform(wordsData)

In [7]:
idf = IDF(inputCol='rawFeatures', outputCol='features')
idfModel = idf.fit(featurizedData)
rescaledData = idfModel.transform(featurizedData)

In [11]:
rescaledData.select("label",'features').show(truncate=False)

+-----+-------------------------------------------------------------------------------------------------------------------------------------------+
|label|features                                                                                                                                   |
+-----+-------------------------------------------------------------------------------------------------------------------------------------------+
|0.0  |(20,[6,8,13,16],[0.28768207245178085,0.6931471805599453,0.28768207245178085,0.5753641449035617])                                           |
|0.0  |(20,[0,2,7,13,15,16],[0.6931471805599453,0.6931471805599453,1.3862943611198906,0.28768207245178085,0.6931471805599453,0.28768207245178085])|
|1.0  |(20,[3,4,6,11,19],[0.6931471805599453,0.6931471805599453,0.28768207245178085,0.6931471805599453,0.6931471805599453])                       |
+-----+---------------------------------------------------------------------------------------------------------

#### Word2Vec

Word2Vec is an Estimator which takes sequences of words representing documents and trains a Word2VecModel. The model maps each word to a unique fixed-size vector. 

The Word2VecModel transforms each document into a vector using the average of all words in the document; this vector can then be used as features for prediction, document similarity calculations, etc.

In [12]:
from pyspark.ml.feature import Word2Vec

In [14]:
# Input data: Each row is a bag of words from a sentence or document.
documentDF = spark.createDataFrame([
    ("Hi I heard about Spark".split(" "), ),
    ("I wish Java could use case classes".split(" "), ),
    ("Logistic regression models are neat".split(" "), )
], ["text"])

documentDF.show(truncate=False)

+------------------------------------------+
|text                                      |
+------------------------------------------+
|[Hi, I, heard, about, Spark]              |
|[I, wish, Java, could, use, case, classes]|
|[Logistic, regression, models, are, neat] |
+------------------------------------------+



In [15]:
word2vec = Word2Vec(vectorSize=3, minCount=0, inputCol='text', outputCol='result')

model = word2vec.fit(documentDF).transform(documentDF)

22/03/29 21:52:27 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
22/03/29 21:52:27 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS


In [18]:
model.show(truncate=False)

+------------------------------------------+-------------------------------------------------------------------+
|text                                      |result                                                             |
+------------------------------------------+-------------------------------------------------------------------+
|[Hi, I, heard, about, Spark]              |[0.10757351368665696,0.005313180014491082,0.02163493409752846]     |
|[I, wish, Java, could, use, case, classes]|[0.02210963943174907,-0.03750888577529362,0.046501401013561657]    |
|[Logistic, regression, models, are, neat] |[-0.023545664548873902,-0.036877965182065965,0.0036725979298353195]|
+------------------------------------------+-------------------------------------------------------------------+



In [24]:
for row in model.collect():
    text, vector = row
    print(f"Text:- {' '.join(text)} => vector:- {vector}")

Text:- Hi I heard about Spark => vector:- [0.10757351368665696,0.005313180014491082,0.02163493409752846]
Text:- I wish Java could use case classes => vector:- [0.02210963943174907,-0.03750888577529362,0.046501401013561657]
Text:- Logistic regression models are neat => vector:- [-0.023545664548873902,-0.036877965182065965,0.0036725979298353195]
